In [ ]:
import os
import sys
import pickle
import gzip

In [ ]:
path = 'NAS-Bench-201-v1_1-096897.pth' # set a path to NAS-Bench-201 benchmark file

from nas_201_api import NASBench201API as API
# Create an API without the verbose log
api = API(path, verbose=False)

In [ ]:
import pickle
obj = pickle.load(gzip.open(open('nasbench_val.pkl.gz', 'rb')))

In [ ]:
hf_val = []
hf_test = []
hf_times = []

lf_val = []
lf_test = []
lf_times = []

for index in range(len(api)):
    #
    # 200 iter
    #
    info = api.query_meta_info_by_index(index, hp = '200')  # This is an instance of `ArchResults`
    
    res_metrics = info.get_metrics('cifar10-valid', 'x-valid', 199)
    #res_metrics = info.get_metrics('cifar100', 'x-valid', 199) 
    #res_metrics = info.get_metrics('ImageNet16-120', 'x-valid', 199) 
    hf_val.append(res_metrics['accuracy'])
    
    res_metrics = info.get_metrics('cifar10', 'ori-test', 199) 
    #res_metrics = info.get_metrics('cifar100', 'x-test', 199)
    #res_metrics = info.get_metrics('ImageNet16-120', 'x-test', 199) 
    hf_test.append(res_metrics['accuracy'])
    
    res_metrics = info.get_metrics('cifar10-valid', 'train', 199)
    #res_metrics = info.get_metrics('cifar100', 'train', 199)
    #res_metrics = info.get_metrics('ImageNet16-120', 'train', 199)
    hf_times.append(res_metrics['all_time'])
    
    #
    # 12 iter
    #
    info = api.query_meta_info_by_index(index, hp = '12')  # This is an instance of `ArchResults`
    res_metrics = info.get_metrics('cifar10-valid', 'x-valid', 11) 
    #res_metrics = info.get_metrics('cifar100', 'x-valid', 11) 
    #res_metrics = info.get_metrics('ImageNet16-120', 'x-valid', 11) 
    lf_val.append(res_metrics['accuracy'])
    
    res_metrics = info.get_metrics('cifar10', 'ori-test', 11)
    #res_metrics = info.get_metrics('cifar100', 'x-test', 11)
    #res_metrics = info.get_metrics('ImageNet16-120', 'x-test', 11) 
    
    lf_test.append(res_metrics['accuracy'])
    
    res_metrics = info.get_metrics('cifar10-valid', 'train', 11) 
    #res_metrics = info.get_metrics('cifar100', 'train', 11)
    #res_metrics = info.get_metrics('ImageNet16-120', 'train', 11)
    lf_times.append(res_metrics['all_time'])

In [ ]:
def print_stat(lf, hf):
    s1 = stats.kendalltau(lf, hf).correlation
    s2 = stats.spearmanr(lf, hf).correlation
    s3 = stats.pearsonr(lf, hf)[0]

    return [s1, s2, s3]

In [ ]:
max(hf_val), max(hf_test)

In [ ]:
len(list(filter(lambda x : x > 93.96, hf_test)))/len(hf_test)*100

In [ ]:
lf_ep1_val = []

for index in range(len(api)):    
    info = api.query_meta_info_by_index(index, hp = '12')  # This is an instance of `ArchResults`
    res_metrics = info.get_metrics('cifar10-valid', 'x-valid', 0)
    #res_metrics = info.get_metrics('cifar100', 'x-valid', 0)
    #res_metrics = info.get_metrics('ImageNet16-120', 'x-valid', 0)
    
    lf_ep1_val.append(res_metrics['accuracy'])

In [ ]:
pickle.dump(lf_ep1_val, open('lf_ep1_val_c10.pickle', 'wb'))
#pickle.dump(lf_ep1_val, open('lf_ep1_val_c100.pickle', 'wb'))
#pickle.dump(lf_ep1_val, open('lf_ep1_val_imagenet.pickle', 'wb'))

In [ ]:
print_stat(lf_ep1_val, hf_test)

In [ ]:
print_stat(lf_nst_ep1, hf_test) 

In [ ]:
pickle.dump((lf_val, lf_test, hf_val, hf_test), open('acc_nb201_c10.pickle', 'wb'))
#pickle.dump((lf_val, lf_test, hf_val, hf_test), open('acc_nb201_c100.pickle', 'wb'))
#pickle.dump((lf_val, lf_test, hf_val, hf_test), open('acc_nb201_imagenet.pickle', 'wb'))